In [11]:
import numpy as np
from torch.nn import TransformerEncoderLayer, TransformerEncoder
from torch.optim import AdamW

Helper functions

In [14]:
def buildContext(D, nb, x, i, M):
    if (len(x[-M:]) < M):
        diff = M - len(x[-M:])
        for i in range(diff):
            D.append(np.ones(nb))
    D.extend(x[-M:])

Inputs

In [13]:
x = [] # Past histograms
y = [] # Ground truth

M = 5 # Buffer size
K = 3 # Batch size

Initialize:

In [12]:
D = []

enc_layer = TransformerEncoderLayer(
    d_model=100,
    nhead=10,
    dim_feedforward=100,
    dropout=0.15
)
enc = TransformerEncoder(
    enc_layer,
    num_layers=3
)

optim = AdamW(enc.parameters(), lr=5e-4, weight_decay=1e-4)
early_stopping_patience = 5

e:\research-paper-experiments\.venv\Lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
for i in range(len(x)):
    nb = len(x[i])

    # Normalize new histogram
    x_t = x[i] / np.sum(x[i])
    
    # Perform inference to obtain reference prediction
    context = buildContext(D, i, M)

    mu_hat_i, sigma_hat_i = enc(context)

    # Compute chi2 here...

    if y[i] == 0:
        D.append((x_t, i))
        # Training step
        batch = D[-K:]
        nll_batch = 0
        optim.zero_grad()
        for (x_k, i_k) in batch:
            context = buildContext(D, i_k, M)
            mu_hat, sigma_hat = enc(context)
            nll_k = computeNLL(x_k, mu_hat, sigma_hat)
            nll_batch += nll_k
        nll_batch = nll_batch / len(batch)
        optim.step()
        # Check early stopping